In [15]:
import torch.nn as nn
import torch

import sys, os
import random
import numpy as np
from shutil import copy
import matplotlib.pyplot as plt
from copy import deepcopy

from omegaconf import OmegaConf
import shutil
import pickle
import random
from tqdm import tqdm

from torchvision.datasets.folder import ImageFolder
from torch.utils.data import DataLoader
# from skimage.filters import threshold_local, gaussian

from datetime import datetime
import pdb

In [38]:
# 154 pruning CUB-18
# run_path = "runs/154-PruningNaiveHPIPNet_cnext13_CUB-18-imgnet-224_with-equalize-aug_img=224_nprotos=20"

# 155 pruning CUB-190
# run_path = "runs/155-PruningNaiveHPIPNet_cnext13_CUB-190-imgnet-224_with-equalize-aug_img=224_nprotos=20"

# 153 pruning CUB-29
# run_path = "runs/153-PruningNaiveHPIPNet_cnext13_CUB-29-imgnet-224_with-equalize-aug_img=224_nprotos=20"

# 156 pruning CUB-18 LOU3
# run_path = "runs/156-PruningNaiveHPIPNet_LOU3_cnext13_CUB-18-imgnet-224_with-equalize-aug_img=224_nprotos=20"

# 157 pruning CUB-29 LOU3
# run_path = "runs/157-PruningNaiveHPIPNet_LOU3_cnext13_CUB-29-imgnet-224_with-equalize-aug_img=224_nprotos=20"

# run_path = "runs/178-PruningNaiveHPIPNetMaskL1=0.5MaskTrainExtra=15epsEps=60TanhDesc=0.05MinCont=0.1_cnext26_CUB-190-imgnet-224_WeightedCE_with-equalize-aug_img=224_nprotos=20"
# ckpt_file_name = 'net_trained_last'

# run_path = "runs/179-PruningNaiveHPIPNetMaskL1=0.5MaskTrainExtra=15epsEps=60TanhDesc=0.05MinCont=0.1_cnext26_CUB-190-224_WeightedCE_with-equalize-aug_img=224_nprotos=20"

# run_path = "runs/180-PruningNaiveHPIPNetMaskL1=0.5MaskTrainExtra=15epsEps=60TanhDesc=2.0MinCont=0.1_cnext26_CUB-190-imgnet-224_WeightedCE_with-equalize-aug_img=224_nprotos=20"

# run_path = "runs/181-PruningNaiveHPIPNetMaskL1=0.5MaskTrainExtra=15epsEps=60TanhDesc=2.0MinCont=0.1_cnext26_CUB-190-224_WeightedCE_with-equalize-aug_img=224_nprotos=20"

# run_path = "runs/182-PruningNaiveHPIPNetMaskL1=0.5MaskTrainExtra=15epsEps=60TanhDesc=0.1MinCont=0.1_cnext26_CUB-190-imgnet-224_WeightedCE_with-equalize-aug_img=224_nprotos=20"

# run_path = "runs/183-PruningBF=1.1NaiveHPIPNetMaskL1=0.5MaskTrainExtra=15epsEps=60TanhDesc=0.05MinCont=0.1_cnext26_CUB-190-imgnet-224_WeightedCE_with-equalize-aug_img=224_nprotos=20"

# run_path = "runs/185-LOUSet1-PruningBF=1.1NaiveHPIPNetMaskL1=0.5MaskTrainExtra=15epsEps=60TanhDesc=0.05MinCont=0.1_cnext26_CUB-190-imgnet-224_WeightedCE_with-equalize-aug_img=224_nprotos=20"

# run_path = "runs/191-LOUSet2-PruningBF=1.1NaiveHPIPNetMaskL1=0.5MaskTrainExtra=05epsEps=85Cl=4.0TanhDesc=0.05MinCont=0.5_cnext26_CUB-190-imgnet-224_WeightedCE_with-equalize-aug_img=224_nprotos=20"

# run_path = "runs/192-PruningNaiveHPIPNetMaskL1=0.5MaskTrainExtra=05epsEps=85Cl=4.0NoTanhDescMinCont=0.5_cnext26_CUB-190-imgnet-224_WeightedCE_with-equalize-aug_img=224_nprotos=20"

# run_path = "runs/193-LOUSet3-PruningBF=1.1NaiveHPIPNetMaskL1=0.5MaskTrainExtra=05epsEps=85Cl=4.0TanhDesc=0.05MinCont=0.5_cnext26_CUB-190-imgnet-224_WeightedCE_with-equalize-aug_img=224_nprotos=20"
# ckpt_file_name = 'net_trained_last'

# run_path = "runs/195-LOUSet3-PruningBF=1.1NaiveHPIPNetMaskL1=0.5MaskTrainExtra=05epsEps=85Cl=4.0TanhDesc=0.05MinCont=0.5_cnext26_CUB-190-imgnet-224_WeightedCE_with-equalize-aug_img=224_nprotos=10pcOr1pd"
# ckpt_file_name = 'net_trained_last'

# run_path = "runs/198-rerun-LOUSet1-PruningBF=1.1NaiveHPIPNetMaskL1=0.5MaskTrainExtra=05epsEps=85Cl=4.0TanhDesc=0.05MinCont=0.5_cnext26_CUB-190-imgnet-224_WeightedCE_with-equalize-aug_img=224_nprotos=10pcOr1pd"
# ckpt_file_name = 'net_trained_last'

# run_path = "runs/199-rerun-LOUSet3-PruningBF=1.1NaiveHPIPNetOODEnt=-0.2MaskL1=0.5MaskTrainExtra=05epsEps=85Cl=4.0TanhDesc=0.05MinCont=0.5_cnext26_CUB-190-imgnet-224_WeightedCE_with-equalize-aug_img=224_nprotos=10pc"
# ckpt_file_name = 'net_trained_90'

# run_path = "runs/197-LOUSet3-PruningBF=1.1NaiveHPIPNetOODEnt=-0.2MaskL1=0.5MaskTrainExtra=05epsEps=85Cl=4.0TanhDesc=0.05MinCont=0.5_cnext26_CUB-190-imgnet-224_WeightedCE_with-equalize-aug_img=224_nprotos=10pcOr1pd"
# ckpt_file_name = 'net_trained_90'

# run_path = "runs/200-rerun-LOUSet1-PruningBF=1.1NaiveHPIPNetOODEnt=-0.2MaskL1=0.5MaskTrainExtra=05epsEps=85Cl=4.0TanhDesc=0.05MinCont=0.5_cnext26_CUB-190-imgnet-224_WeightedCE_with-equalize-aug_img=224_nprotos=10pc"
# ckpt_file_name = 'net_trained_last'

# run_path = "runs/201-LOUSet1-PruningBF=1.1NaiveHPIPNetOODEnt=-0.2MaskL1=0.5MaskTrainExtra=05epsEps=85Cl=4.0TanhDesc=0.05MinCont=0.5_cnext26_CUB-190-imgnet-224_WeightedCE_with-equalize-aug_img=224_nprotos=10pcOr1pd"
# ckpt_file_name = 'net_trained_90'

# run_path = "runs/205-LOUSet4-PruningBF=1.1NaiveHPIPNetMaskL1=0.5MaskTrainExtra=05epsEps=60Cl=2.0TanhDesc=0.05MinCont=0.1_cnext26_CUB-190-imgnet-224_WeightedCE_with-equalize-aug_img=224_nprotos=10pc"
# ckpt_file_name = 'net_trained_last'

# run_path = "runs/206-LOUSet5-PruningBF=1.1NaiveHPIPNetMaskL1=0.5MaskTrainExtra=05epsEps=60Cl=2.0TanhDesc=0.05MinCont=0.1_cnext26_CUB-190-imgnet-224_WeightedCE_with-equalize-aug_img=224_nprotos=10pc"
# ckpt_file_name = 'net_trained_last'

# run_path = "runs/207-PruningBF=1.1NaiveHPIPNetMaskL1=0.5MaskTrainExtra=05epsEps=60Cl=2.0TanhDesc=0.05MinCont=0.1_cnext26_BUT-51-224_WeightedCE_with-equalize-aug_img=224_nprotos=10pc"
# ckpt_file_name = 'net_trained_last'

run_path = "runs/208-PruningBF=1.1NaiveHPIPNetMaskL1=0.5MaskTrainExtra=05epsEps=60Cl=2.0TanhDesc=0.05MinCont=0.1_cnext26_FISH-38-224_WeightedCE_with-equalize-aug_img=224_nprotos=10pc"
ckpt_file_name = 'net_trained_last'


# try:
#     sys.path.remove('/home/harishbabu/projects/PIPNet')
# except:
#     pass
# sys.path.insert(0, os.path.join(run_path, 'source_clone'))

print(run_path)

runs/208-PruningBF=1.1NaiveHPIPNetMaskL1=0.5MaskTrainExtra=05epsEps=60Cl=2.0TanhDesc=0.05MinCont=0.1_cnext26_FISH-38-224_WeightedCE_with-equalize-aug_img=224_nprotos=10pc


In [39]:
from pipnet.pipnet import PIPNet, get_network
from util.log import Log
from util.args import get_args, save_args, get_optimizer_nn
from util.data import get_dataloaders
from util.func import init_weights_xavier
from pipnet.train import train_pipnet, test_pipnet
# from pipnet.test import eval_pipnet, get_thresholds, eval_ood
from util.eval_cub_csv import eval_prototypes_cub_parts_csv, get_topk_cub, get_proto_patches_cub
from util.vis_pipnet import visualize, visualize_topk
from util.visualize_prediction import vis_pred, vis_pred_experiments
from util.node import Node
from util.phylo_utils import construct_phylo_tree, construct_discretized_phylo_tree
from util.func import get_patch_size
from util.data import ModifiedLabelLoader

# Save leave_out_classes to a variable

In [40]:
args_file = open(os.path.join(run_path, 'metadata', 'args.pickle'), 'rb')
args = pickle.load(args_file)

if ('leave_out_classes' in args) and (args.leave_out_classes != ''):
        with open(args.leave_out_classes, 'r') as file:
            leave_out_classes = [line.strip() for line in file]

# Define tree

In [41]:
args_file = open(os.path.join(run_path, 'metadata', 'args.pickle'), 'rb')
args = pickle.load(args_file)

if args.phylo_config:
    phylo_config = OmegaConf.load(args.phylo_config)

if args.phylo_config:
    # construct the phylo tree
    if phylo_config.phyloDistances_string == 'None':
        if '031' in run_path: # this run uses a different phylogeny file that had an extra root node which is a mistake
            root = construct_phylo_tree('/home/harishbabu/data/phlyogenyCUB/18Species-with-extra-root-node/1_tree-consensus-Hacket-18Species-modified_cub-names_v1.phy')
        else:
            root = construct_phylo_tree(phylo_config.phylogeny_path)
        print('-'*25 + ' No discretization ' + '-'*25)
    else:
        root = construct_discretized_phylo_tree(phylo_config.phylogeny_path, phylo_config.phyloDistances_string)
        print('-'*25 + ' Discretized ' + '-'*25)
else:
    # construct the tree (original hierarchy as described in the paper)
    root = Node("root")
    root.add_children(['animal','vehicle','everyday_object','weapon','scuba_diver'])
    root.add_children_to('animal',['non_primate','primate'])
    root.add_children_to('non_primate',['African_elephant','giant_panda','lion'])
    root.add_children_to('primate',['capuchin','gibbon','orangutan'])
    root.add_children_to('vehicle',['ambulance','pickup','sports_car'])
    root.add_children_to('everyday_object',['laptop','sandal','wine_bottle'])
    root.add_children_to('weapon',['assault_rifle','rifle'])
    # flat root
    # root.add_children(['scuba_diver','African_elephant','giant_panda','lion','capuchin','gibbon','orangutan','ambulance','pickup','sports_car','laptop','sandal','wine_bottle','assault_rifle','rifle'])
root.assign_all_descendents()

exp_no = int(os.path.basename(run_path)[:3])

if exp_no < 77:
    if ('num_protos_per_descendant' in args) and (args.num_protos_per_descendant > 0):
        for node in root.nodes_with_children():
            node.set_num_protos(args.num_protos_per_descendant)
if exp_no == 77:
    # update num of protos per node based on num_protos_per_descendant
    if args.num_features == 0 and args.num_protos_per_descendant == 0:
        raise Exception('Either of num_features or num_protos_per_descendant must be greater than zero')
    for node in root.nodes_with_children():
        node.set_num_protos(num_protos_per_descendant=args.num_protos_per_descendant,\
                                                            min_protos=args.num_features)

elif 'num_protos_per_child' in args:
    if ('num_protos_per_descendant' in args):
        # update num of protos per node based on num_protos_per_descendant
        if args.num_features == 0 and args.num_protos_per_descendant == 0 and args.num_protos_per_child == 0:
            raise Exception('Either of num_features or num_protos_per_descendant must be greater than zero')
        for node in root.nodes_with_children():
            node.set_num_protos(num_protos_per_descendant=args.num_protos_per_descendant,\
                                num_protos_per_child=args.num_protos_per_child,\
                                min_protos=args.num_features,\
                                split_protos=('protopool' in args) and (args.protopool == 'n'))
else:
    if ('num_protos_per_descendant' in args):
        # update num of protos per node based on num_protos_per_descendant
        if args.num_features == 0 and args.num_protos_per_descendant == 0:
            raise Exception('Either of num_features or num_protos_per_descendant must be greater than zero')
        for node in root.nodes_with_children():
            node.set_num_protos(num_protos_per_descendant=args.num_protos_per_descendant,\
                                num_protos_per_child=0,\
                                min_protos=args.num_features,\
                                split_protos=('protopool' in args) and (args.protopool == 'n'))

FileNotFoundError: [Errno 2] No such file or directory: '/home/harishbabu/projects/PIPNet/configs/fish38_phylogeny.yaml'

# Load model

In [42]:
if torch.cuda.is_available():
    device = torch.device('cuda')
    device_ids = [torch.cuda.current_device()]
else:
    device = torch.device('cpu')
    device_ids = []

# args_file = open(os.path.join(run_path, 'metadata', 'args.pickle'), 'rb')
# args = pickle.load(args_file)

# ckpt_file_name = 'net_overspecific_pruned_replaced_thresh=0.5_last'
# ckpt_file_name = 'net_trained_last'
# ckpt_file_name = 'net_trained_10'
# ckpt_file_name = 'net_pretrained'
epoch = ckpt_file_name.split('_')[-1]

ckpt_path = os.path.join(run_path, 'checkpoints', ckpt_file_name)
checkpoint = torch.load(ckpt_path, map_location=device)

if ckpt_file_name != 'net_trained_last':
    print('\n', (10*'-')+'WARNING: Not using the final trained model'+(10*'-'), '\n')

# Obtain the dataset and dataloaders
temp = args.leave_out_classes
args.leave_out_classes = '' # NOTE: because here we need to load the entire dataloader, and filter out the classes later
trainloader, trainloader_pretraining, trainloader_normal, trainloader_normal_augment, projectloader, testloader, test_projectloader, classes = get_dataloaders(args, device)
args.leave_out_classes = temp

print(args.batch_size, trainloader.batch_size)

if len(classes)<=20:
    if args.validation_size == 0.:
        print("Classes: ", testloader.dataset.class_to_idx, flush=True)
    else:
        print("Classes: ", str(classes), flush=True)

# Create a convolutional network based on arguments and add 1x1 conv layer
feature_net, add_on_layers, pool_layer, classification_layers, num_prototypes = get_network(len(classes), args, root=root)
   
# Create a PIP-Net
net = PIPNet(num_classes=len(classes),
                    num_prototypes=num_prototypes,
                    feature_net = feature_net,
                    args = args,
                    add_on_layers = add_on_layers,
                    pool_layer = pool_layer,
                    classification_layers = classification_layers,
                    num_parent_nodes = len(root.nodes_with_children()),
                    root = root
                    )
net = net.to(device=device)
net = nn.DataParallel(net, device_ids = device_ids)    
net.load_state_dict(checkpoint['model_state_dict'],strict=True)
# print(net.eval())
criterion = nn.NLLLoss(reduction='mean').to(device)

# Forward one batch through the backbone to get the latent output size
with torch.no_grad():
    xs1, _, _ = next(iter(trainloader))
    xs1 = xs1.to(device)
    _, proto_features, _, _ = net(xs1)
    wshape = proto_features['root'].shape[-1]
    args.wshape = wshape #needed for calculating image patch size
    print("Output shape: ", proto_features['root'].shape, flush=True)
    
print(args.wshape)

Num classes (k) =  38 ['fis_001_Alosa_chrysochloris', 'fis_002_Carassius_auratus', 'fis_003_Cyprinus_carpio', 'fis_004_Esox_americanus', 'fis_005_Gambusia_affinis'] etc.
64 64
Number of prototypes:  768
----------Prototypes per descendant: 0----------
Assigned 20 protos to node root
Assigned 20 protos to node 129+024+067
Assigned 20 protos to node 089+046
Assigned 20 protos to node 129+065
Assigned 20 protos to node 024+051
Assigned 20 protos to node 067+070
Assigned 20 protos to node 089+090
Assigned 20 protos to node 046+087
Assigned 20 protos to node 129+192
Assigned 20 protos to node 065+006
Assigned 20 protos to node 024+031
Assigned 20 protos to node 051+052
Assigned 20 protos to node 067+068
Assigned 20 protos to node 129+043
Assigned 20 protos to node 192+081
Assigned 20 protos to node 065+144
Assigned 20 protos to node 006+071
Assigned 20 protos to node 024+086
Assigned 20 protos to node 031+004
Assigned 20 protos to node 051+053
Assigned 20 protos to node 067+069
Assigned 20 

RuntimeError: Error(s) in loading state_dict for DataParallel:
	Missing key(s) in state_dict: "module._129+024+067_proto_presence", "module._089+046_proto_presence", "module._129+065_proto_presence", "module._024+051_proto_presence", "module._067+070_proto_presence", "module._089+090_proto_presence", "module._046+087_proto_presence", "module._129+192_proto_presence", "module._065+006_proto_presence", "module._024+031_proto_presence", "module._051+052_proto_presence", "module._067+068_proto_presence", "module._129+043_proto_presence", "module._192+081_proto_presence", "module._065+144_proto_presence", "module._006+071_proto_presence", "module._024+086_proto_presence", "module._031+004_proto_presence", "module._051+053_proto_presence", "module._067+069_proto_presence", "module._129+018_proto_presence", "module._043+078_proto_presence", "module._192+036_proto_presence", "module._081+083_proto_presence", "module._065+084_proto_presence", "module._144+147_proto_presence", "module._006+058_p

In [ ]:
import wandb
from util.log import Log
os.environ['WANDB_DISABLED'] = 'true'
wandb_run = wandb.init(project="pipnet", name=os.path.basename(args.log_dir), config=vars(args), reinit=False)
log = Log(args.log_dir)

In [43]:
optimizer_net, optimizer_classifier, params_to_freeze, params_to_train, params_backbone = get_optimizer_nn(net, args)            
scheduler_net = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer_net, T_max=len(trainloader)*args.epochs, eta_min=args.lr_net/100.)
if args.epochs<=30:
    scheduler_classifier = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer_classifier, T_0=5, eta_min=0.001, T_mult=1, verbose=False)
else:
    scheduler_classifier = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer_classifier, T_0=10, eta_min=0.001, T_mult=1, verbose=False)

    
if args.OOD_dataset:
    trainloader_OOD, trainloader_pretraining_OOD, trainloader_normal_OOD, trainloader_normal_augment_OOD, projectloader_OOD, testloader_OOD, test_projectloader_OOD, _ = get_dataloaders(args, device, OOD=True)
    print('-'*25 + 'Using OOD data' + '-'*25)
else:
    trainloader_OOD = trainloader_pretraining_OOD = trainloader_normal_OOD = trainloader_normal_augment_OOD = projectloader_OOD = testloader_OOD = test_projectloader_OOD = None
    print('-'*25 + 'Not using OOD data' + '-'*25)
    
if ('focal_loss' in args) and (args.focal_loss == 'y'):
    from util.custom_losses import WeightedCrossEntropyLoss, WeightedNLLLoss, FocalLossWrapper
    criterion = FocalLossWrapper(device=device, alpha=1, gamma=args.focal_loss_gamma, reduction='mean').to(device)
else:
    from util.custom_losses import WeightedCrossEntropyLoss, WeightedNLLLoss, FocalLossWrapper
    criterion = WeightedNLLLoss(device=device).to(device)

chosen network is convnext
-------------------------Not using OOD data-------------------------


# Helper functions

In [9]:
def get_heatmap(latent_activation, input_image):
    image_a = latent_activation.cpu().numpy()
    image_a = (image_a - image_a.min()) / (image_a.max() - image_a.min())

    input_image = (input_image - input_image.min()) / (input_image.max() - input_image.min())
    image_b = input_image.permute(1, 2, 0).cpu().numpy()
    
    reshaped_image_a = np.array(Image.fromarray((image_a[0] * 255).astype('uint8')).resize((input_image.shape[-1], input_image.shape[-1])))
    normalized_heatmap = (reshaped_image_a - np.min(reshaped_image_a)) / (np.max(reshaped_image_a) - np.min(reshaped_image_a))
    
    heatmap_colormap = plt.get_cmap('jet')
    heatmap_colored = heatmap_colormap(normalized_heatmap)
    
    heatmap_colored_uint8 = (heatmap_colored[:, :, :3] * 255).astype(np.uint8)
    image_a_heatmap_pillow = Image.fromarray(heatmap_colored_uint8)
    image_b_pillow = Image.fromarray((image_b * 255).astype('uint8'))
    
    result_image = Image.blend(image_b_pillow, image_a_heatmap_pillow, alpha=0.3)
    
    return np.array(result_image)


def get_heatmap_uninterpolated(latent_activation, input_image):
    image_a = latent_activation.cpu().numpy()
    image_a = (image_a - image_a.min()) / (image_a.max() - image_a.min())

    input_image = (input_image - input_image.min()) / (input_image.max() - input_image.min())
    image_b = input_image.permute(1, 2, 0).cpu().numpy()
    
    reshaped_image_a = np.array(Image.fromarray((image_a[0] * 255).astype('uint8')).resize((input_image.shape[-1], input_image.shape[-1]), \
                                                                                          resample=Image.NEAREST ))
    normalized_heatmap = (reshaped_image_a - np.min(reshaped_image_a)) / (np.max(reshaped_image_a) - np.min(reshaped_image_a))
    
    heatmap_colormap = plt.get_cmap('jet')
    heatmap_colored = heatmap_colormap(normalized_heatmap)
    
    heatmap_colored_uint8 = (heatmap_colored[:, :, :3] * 255).astype(np.uint8)
    image_a_heatmap_pillow = Image.fromarray(heatmap_colored_uint8)
    image_b_pillow = Image.fromarray((image_b * 255).astype('uint8'))
    
    result_image = Image.blend(image_b_pillow, image_a_heatmap_pillow, alpha=0.3)
    
    return np.array(result_image)

def get_bb_gaussian_threshold(latent_activation, sigma=1.0, percentile=97, extend_h=0, extend_w=0):
    # latent_activation -> []
    upscaled_similarity = get_upscaled_activation_uninterpolated(latent_activation, \
                                                                 image_size=(args.image_size, args.image_size))
    upscaled_similarity = minmaxscale(upscaled_similarity)
    upscaled_similarity = gaussian(upscaled_similarity, sigma=sigma)
    upscaled_similarity = threshold_local(upscaled_similarity, block_size=15, method='mean')
    h_min, h_max, w_min, w_max = find_top_percentile_bbox(upscaled_similarity ,percentile=97)
    h_min = max(0, h_min-extend_h)
    h_max = min(upscaled_similarity.shape[0], h_max+extend_h)
    w_min = max(0, w_min-extend_w)
    w_max = min(upscaled_similarity.shape[1], w_max+extend_w)
    return h_min, h_max, w_min, w_max


def minmaxscale(tensor):
    return (tensor - tensor.min()) / (tensor.max() - tensor.min())

from torch.utils.data import DataLoader, SequentialSampler
from torch.utils.data import DataLoader

def unshuffle_dataloader(dataloader):
    if type(dataloader.dataset) == ImageFolder:
        dataset = dataloader.dataset
    else:
        dataset = dataloader.dataset.dataset.dataset
    new_dataloader = DataLoader(
        dataset=dataset,
        batch_size=dataloader.batch_size,
        shuffle=False,
        num_workers=dataloader.num_workers,
        pin_memory=dataloader.pin_memory,
        drop_last=dataloader.drop_last,
        timeout=dataloader.timeout,
        worker_init_fn=dataloader.worker_init_fn,
        multiprocessing_context=dataloader.multiprocessing_context,
        generator=dataloader.generator,
        prefetch_factor=dataloader.prefetch_factor,
        persistent_workers=dataloader.persistent_workers
    )
    return new_dataloader

# Fine accuracy

In [44]:
if 'cl_weight' not in args:
    args.cl_weight = 2.0

In [45]:
# testloader

test_info, log_dict = test_pipnet(net, testloader, optimizer_net, optimizer_classifier, \
                                    scheduler_net, scheduler_classifier, criterion, 0, \
                                        args.epochs, device, pretrain=False, finetune=False, \
                                        test_loader_OOD=testloader_OOD, kernel_orth=args.kernel_orth == 'y', \
                                            tanh_desc=args.tanh_desc == 'y', align=args.align == 'y', uni=args.uni == 'y', align_pf=args.align_pf == 'y',\
                                            minmaximize=args.minmaximize == 'y', wandb_run=wandb_run, pretrain_epochs=args.epochs_pretrain, log=log, \
                                          args=args, apply_overspecificity_mask=False, path_prob_softmax_tau=1)

Test Epoch0:   0% 0/21 [00:01<?, ?it/s]


AttributeError: 'int' object has no attribute 'item'

In [ ]:
round(test_info['fine_accuracy'], 4)
test_info['fine_accuracy']

# Geometric mean accuracy

In [30]:
leafname_to_depth = {}
leafname_to_idx = testloader.dataset.class_to_idx



def get_node_depth(node):
    depth = 0
    while node.parent:
        depth += 1
        node = node.parent
    return depth

for leaf_name in root.leaf_descendents:
    leaf_node = root.get_node(leaf_name)
    leafname_to_depth[leaf_name] = get_node_depth(leaf_node)

def get_path_prob_list(out_probs, data_idx, root, leaf_node):
    node = leaf_node
    path_prob_list = []
    while node.parent:
        parent = node.parent
        child_class_idx = parent.children_to_labels[node.name]
        path_prob_list.append(out_probs[parent.name][data_idx, child_class_idx])
        node = parent
    return path_prob_list

correct = 0
total = 0

with torch.no_grad():
    for i, (xs, ys) in tqdm(enumerate(testloader), total=len(testloader)):

        features, proto_features, pooled, out = net(xs)
        out_probs = {}
        for node_name, output_logits in out.items():
            out_probs[node_name] = torch.softmax(output_logits, dim=1)

        batch_leaf_probs_list = []
        for data_idx in range(xs.shape[0]):
            leaf_probs = [0 for _ in range(len(leafname_to_idx))]
            for leafname, class_idx in leafname_to_idx.items():
                leaf_node = root.get_node(leafname)
                path_prob_list = get_path_prob_list(out_probs, data_idx, root, leaf_node)
                depth = leafname_to_depth[leafname]
                leaf_probs[class_idx] = torch.tensor([prob**(1/depth) for prob in path_prob_list]).prod()
            batch_leaf_probs_list.append(torch.tensor(leaf_probs))
        batch_leaf_probs = torch.stack(batch_leaf_probs_list)

        correct += (batch_leaf_probs.argmax(dim=1) == ys).sum()
        total += ys.shape[0]

print('Accuracy:', correct / total)

100%|██████████| 22/22 [07:48<00:00, 21.29s/it]

Accuracy: tensor(0.7003)


In [29]:
len(testloader)

22

# Leave one out accuracy

## Remove classes from dataloader

In [10]:
from torch.utils.data import Sampler, SubsetRandomSampler

def create_filtered_dataloader(dataloader, new_sampler, batch_size):
    if type(dataloader.dataset) == ImageFolder:
        dataset = dataloader.dataset
    else:
        dataset = dataloader.dataset.dataset.dataset
    new_dataloader = DataLoader(
        dataset=dataset,
        batch_size=batch_size,
        shuffle=False,
        sampler=new_sampler,
        num_workers=dataloader.num_workers,
        pin_memory=dataloader.pin_memory,
        drop_last=dataloader.drop_last,
        timeout=dataloader.timeout,
        worker_init_fn=dataloader.worker_init_fn,
        multiprocessing_context=dataloader.multiprocessing_context,
        generator=dataloader.generator,
        prefetch_factor=dataloader.prefetch_factor,
        persistent_workers=dataloader.persistent_workers
    )
    return new_dataloader

leave_out_loader = testloader
# leave_out_loader = test_projectloader
unique_labels = set()
total_samples = 0
for xs, ys in testloader:
    unique_labels.update(ys.tolist())
    total_samples += xs.shape[0]
print("testloader Unique Labels:", unique_labels)
print("testloader total_samples:", total_samples)

# leave_out_classes = args.leave_out_classes.split(',')[2]

class_of_interest = 7
classes_to_keep = [leave_out_classes[class_of_interest]]

# classes_to_keep = [leave_out_classes[1], leave_out_classes[6], leave_out_classes[8]]
# classes_to_keep = [leave_out_classes[0], leave_out_classes[1], leave_out_classes[2], leave_out_classes[3], leave_out_classes[6], leave_out_classes[7]]


# classes_to_keep = leave_out_classes

# leave_out_classes = ['cub_093_Clark_Nutcracker']

print('classes_to_keep', classes_to_keep)
idx_of_classes_to_keep = set()
name2label = leave_out_loader.dataset.class_to_idx # param
label2name = {label:name for name, label in name2label.items()}
for label in label2name:
    # NOTE: Keeping the left out classes here
    if label2name[label] in classes_to_keep:
        idx_of_classes_to_keep.add(label)

target_indices = []
for i in range(len(leave_out_loader.dataset)):
    *_, label = leave_out_loader.dataset[i]
    if label in idx_of_classes_to_keep:
        target_indices.append(i)
sampler = SubsetRandomSampler(target_indices)
to_shuffle = False
    
leave_out_loader = create_filtered_dataloader(leave_out_loader, sampler, batch_size=256) # dataloader.batch_size
unique_labels = set()
total_samples = 0
for xs, ys in leave_out_loader:
    unique_labels.update(ys.tolist())
    total_samples += xs.shape[0]
print("leave_out_loader Unique Labels:", unique_labels)
print("leave_out_loader total_samples:", total_samples)

name2label = leave_out_loader.dataset.class_to_idx
label2name = {label:name for name, label in name2label.items()}

# print('Leave out classes', args.leave_out_classes)

testloader Unique Labels: {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189}
testloader total_samples: 5512
classes_to_keep ['cub_112_Great_Grey_Shrike']
leave_out_loader Unique Labels: {104}
leave_out_loader t

In [11]:
# leave_out_loader.dataset.class_to_idx

In [12]:
if 'cl_weight' not in args:
    args.cl_weight = 2.0

## Leave out without mask pruning

In [13]:
# testloader

test_info, log_dict = test_pipnet(net, leave_out_loader, optimizer_net, optimizer_classifier, \
                                    scheduler_net, scheduler_classifier, criterion, 0, \
                                        args.epochs, device, pretrain=False, finetune=False, \
                                        test_loader_OOD=testloader_OOD, kernel_orth=args.kernel_orth == 'y', \
                                            tanh_desc=args.tanh_desc == 'y', align=args.align == 'y', uni=args.uni == 'y', align_pf=args.align_pf == 'y',\
                                            minmaximize=args.minmaximize == 'y', wandb_run=wandb_run, pretrain_epochs=args.epochs_pretrain, log=log, \
                                          args=args, apply_overspecificity_mask=False, leave_out_classes=leave_out_classes, path_prob_softmax_tau=1)

Test Epoch0: 100% 1/1 [00:00<00:00,  1.32it/s, L:0.146,LC:0.760, L_OVSP:-0.004, L_MASKL1:0.001, LA_PF:0.011, LT:-5.000, L_ORTH:0.001, LT_DESC:-5.000, L_OOD_ENT:-5.000, losses_used:MASK_PRUNING+MIN_CONT+AL_PF+KO+CL]
/home/harishbabu/.conda/envs/hpnet4/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/harishbabu/.conda/envs/hpnet4/lib/python3.9/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


	Fine accuracy: 0.57
	Node name: root, acc: 100.0, f1:100.0, samples: 60, 129+024+067=60/60=1.0, 089+046=0/0=nan
	Node name: 129+024+067, acc: 86.67, f1:92.86, samples: 60, 129+065=52/60=0.87, 024+051=0/0=nan, 067+070=0/0=nan
	Node name: 129+065, acc: 100.0, f1:100.0, samples: 60, 129+192=60/60=1.0, 065+006=0/0=nan
	Node name: 129+192, acc: 100.0, f1:100.0, samples: 60, 129+043=60/60=1.0, 192+081=0/0=nan
	Node name: 129+043, acc: 70.0, f1:82.35, samples: 60, 129+018=42/60=0.7, 043+078=0/0=nan
	Node name: 129+018, acc: 100.0, f1:100.0, samples: 60, 129+107=60/60=1.0, cub_018_Spotted_Catbird=0/0=nan
	Node name: 129+107, acc: 96.67, f1:98.31, samples: 60, 129+136=0/0=nan, 107+151=58/60=0.97
	Node name: 107+151, acc: 100.0, f1:100.0, samples: 60, 107+111=60/60=1.0, 151+153=0/0=nan
	Node name: 107+111, acc: 96.67, f1:98.31, samples: 60, 107+073=0/0=nan, 111+112=58/60=0.97
	Node name: 111+112, acc: 0.0, f1:0.0, samples: 60, cub_111_Loggerhead_Shrike=0/0=nan, cub_112_Great_Grey_Shrike=0/60=0.

In [18]:
round(test_info['fine_accuracy'], 2)
leave_out_classes

['cub_030_Fish_Crow',
 'cub_198_Rock_Wren',
 'cub_147_Least_Tern',
 'cub_014_Indigo_Bunting',
 'cub_083_White_breasted_Kingfisher',
 'cub_138_Tree_Swallow',
 'cub_185_Bohemian_Waxwing',
 'cub_112_Great_Grey_Shrike']

In [60]:
# # test_info['node_accuracy'].values()

# # print(leave_out_classes[class_of_interest])

# # for key in test_info['node_accuracy']:
# #     if leave_out_classes[class_of_interest] in modified_root.get_node(key).leaf_descendents:
# #         print(key, test_info['node_accuracy'][key]['accuracy'])

# print(leave_out_classes[class_of_interest])
# leave_out_accuracy = 1
# for key in test_info['node_accuracy']:
#     if leave_out_classes[class_of_interest] in root.get_node(key).leaf_descendents:
#         if not (leave_out_classes[class_of_interest] in [child.name for child in root.get_node(key).children]):
#             leave_out_accuracy *= test_info['node_accuracy'][key]['accuracy'] / 100
#         print(key, test_info['node_accuracy'][key]['accuracy'])

# print(leave_out_accuracy)

In [10]:
from torch.utils.data import Sampler, SubsetRandomSampler

def create_filtered_dataloader(dataloader, new_sampler, batch_size):
    if type(dataloader.dataset) == ImageFolder:
        dataset = dataloader.dataset
    else:
        dataset = dataloader.dataset.dataset.dataset
    new_dataloader = DataLoader(
        dataset=dataset,
        batch_size=batch_size,
        shuffle=False,
        sampler=new_sampler,
        num_workers=dataloader.num_workers,
        pin_memory=dataloader.pin_memory,
        drop_last=dataloader.drop_last,
        timeout=dataloader.timeout,
        worker_init_fn=dataloader.worker_init_fn,
        multiprocessing_context=dataloader.multiprocessing_context,
        generator=dataloader.generator,
        prefetch_factor=dataloader.prefetch_factor,
        persistent_workers=dataloader.persistent_workers
    )
    return new_dataloader

leave_out_loader = testloader

for left_out_class in leave_out_classes:

    classes_to_keep = [left_out_class]
    
    print('classes_to_keep', classes_to_keep)
    idx_of_classes_to_keep = set()
    name2label = leave_out_loader.dataset.class_to_idx # param
    label2name = {label:name for name, label in name2label.items()}
    for label in label2name:
        # NOTE: Keeping the left out classes here
        if label2name[label] in classes_to_keep:
            idx_of_classes_to_keep.add(label)
    
    target_indices = []
    for i in range(len(leave_out_loader.dataset)):
        *_, label = leave_out_loader.dataset[i]
        if label in idx_of_classes_to_keep:
            target_indices.append(i)
    sampler = SubsetRandomSampler(target_indices)
    to_shuffle = False
        
    leave_out_loader = create_filtered_dataloader(leave_out_loader, sampler, batch_size=256) # dataloader.batch_size
    unique_labels = set()
    total_samples = 0
    for xs, ys in leave_out_loader:
        unique_labels.update(ys.tolist())
        total_samples += xs.shape[0]
    print("leave_out_loader Unique Labels:", unique_labels)
    print("leave_out_loader total_samples:", total_samples)
    
    name2label = leave_out_loader.dataset.class_to_idx
    label2name = {label:name for name, label in name2label.items()}
    
    # print('Leave out classes', args.leave_out_classes)

    # testloader

    test_info, log_dict = test_pipnet(net, leave_out_loader, optimizer_net, optimizer_classifier, \
                                        scheduler_net, scheduler_classifier, criterion, 0, \
                                            args.epochs, device, pretrain=False, finetune=False, \
                                            test_loader_OOD=testloader_OOD, kernel_orth=args.kernel_orth == 'y', \
                                                tanh_desc=args.tanh_desc == 'y', align=args.align == 'y', uni=args.uni == 'y', align_pf=args.align_pf == 'y',\
                                                minmaximize=args.minmaximize == 'y', wandb_run=wandb_run, pretrain_epochs=args.epochs_pretrain, log=log, \
                                              args=args, apply_overspecificity_mask=False, leave_out_classes=classes_to_keep, path_prob_softmax_tau=1)
    print(left_out_class, round(test_info['fine_accuracy'], 2), '-'*10)
    print('\n')

classes_to_keep ['cub_030_Fish_Crow']
leave_out_loader Unique Labels: {28}
leave_out_loader total_samples: 30


Test Epoch0: 100% 1/1 [00:00<00:00,  1.82it/s, L:0.039,LC:0.626, L_OVSP:-0.006, L_MASKL1:0.001, LA_PF:0.012, LT:-5.000, L_ORTH:0.005, LT_DESC:-5.000, L_OOD_ENT:-5.000, losses_used:MASK_PRUNING+MIN_CONT+AL_PF+KO+CL]
/home/harishbabu/.conda/envs/hpnet4/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/harishbabu/.conda/envs/hpnet4/lib/python3.9/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


	Fine accuracy: 0.5333
	Node name: root, acc: 96.67, f1:98.31, samples: 60, 129+024+067=58/60=0.97, 089+046=0/0=nan
	Node name: 129+024+067, acc: 63.33, f1:77.55, samples: 60, 129+065=38/60=0.63, 024+051=0/0=nan, 067+070=0/0=nan
	Node name: 129+065, acc: 86.67, f1:92.86, samples: 60, 129+192=52/60=0.87, 065+006=0/0=nan
	Node name: 129+192, acc: 100.0, f1:100.0, samples: 60, 129+043=60/60=1.0, 192+081=0/0=nan
	Node name: 129+043, acc: 100.0, f1:100.0, samples: 60, 129+018=60/60=1.0, 043+078=0/0=nan
	Node name: 129+018, acc: 96.67, f1:98.31, samples: 60, 129+107=58/60=0.97, cub_018_Spotted_Catbird=0/0=nan
	Node name: 129+107, acc: 83.33, f1:90.91, samples: 60, 129+136=0/0=nan, 107+151=50/60=0.83
	Node name: 107+151, acc: 100.0, f1:100.0, samples: 60, 107+111=60/60=1.0, 151+153=0/0=nan
	Node name: 107+111, acc: 100.0, f1:100.0, samples: 60, 107+073=60/60=1.0, 111+112=0/0=nan
	Node name: 107+073, acc: 100.0, f1:100.0, samples: 60, 107+093=60/60=1.0, 073+074=0/0=nan
	Node name: 107+093, acc

Test Epoch0: 100% 1/1 [00:00<00:00,  1.84it/s, L:0.044,LC:0.554, L_OVSP:-0.005, L_MASKL1:0.001, LA_PF:0.010, LT:-5.000, L_ORTH:0.005, LT_DESC:-5.000, L_OOD_ENT:-5.000, losses_used:MASK_PRUNING+MIN_CONT+AL_PF+KO+CL]


	Fine accuracy: 0.5333
	Node name: root, acc: 100.0, f1:100.0, samples: 60, 129+024+067=60/60=1.0, 089+046=0/0=nan
	Node name: 129+024+067, acc: 83.33, f1:90.91, samples: 60, 129+065=50/60=0.83, 024+051=0/0=nan, 067+070=0/0=nan
	Node name: 129+065, acc: 100.0, f1:100.0, samples: 60, 129+192=60/60=1.0, 065+006=0/0=nan
	Node name: 129+192, acc: 100.0, f1:100.0, samples: 60, 129+043=60/60=1.0, 192+081=0/0=nan
	Node name: 129+043, acc: 90.0, f1:94.74, samples: 60, 129+018=54/60=0.9, 043+078=0/0=nan
	Node name: 129+018, acc: 100.0, f1:100.0, samples: 60, 129+107=60/60=1.0, cub_018_Spotted_Catbird=0/0=nan
	Node name: 129+107, acc: 100.0, f1:100.0, samples: 60, 129+136=60/60=1.0, 107+151=0/0=nan
	Node name: 129+136, acc: 100.0, f1:100.0, samples: 60, 129+199=60/60=1.0, 136+085=0/0=nan
	Node name: 129+199, acc: 80.0, f1:88.89, samples: 60, 129+118=0/0=nan, 199+186=48/60=0.8
	Node name: 199+186, acc: 100.0, f1:100.0, samples: 60, 199+150=60/60=1.0, 186+185=0/0=nan
	Node name: 199+150, acc: 76.6

Test Epoch0: 100% 1/1 [00:00<00:00,  1.84it/s, L:0.025,LC:0.409, L_OVSP:-0.004, L_MASKL1:0.001, LA_PF:0.008, LT:-5.000, L_ORTH:0.004, LT_DESC:-5.000, L_OOD_ENT:-5.000, losses_used:MASK_PRUNING+MIN_CONT+AL_PF+KO+CL]


	Fine accuracy: 0.9667
	Node name: root, acc: 100.0, f1:100.0, samples: 60, 129+024+067=60/60=1.0, 089+046=0/0=nan
	Node name: 129+024+067, acc: 100.0, f1:100.0, samples: 60, 129+065=60/60=1.0, 024+051=0/0=nan, 067+070=0/0=nan
	Node name: 129+065, acc: 100.0, f1:100.0, samples: 60, 129+192=60/60=1.0, 065+006=0/0=nan
	Node name: 129+192, acc: 100.0, f1:100.0, samples: 60, 129+043=60/60=1.0, 192+081=0/0=nan
	Node name: 129+043, acc: 100.0, f1:100.0, samples: 60, 129+018=60/60=1.0, 043+078=0/0=nan
	Node name: 129+018, acc: 100.0, f1:100.0, samples: 60, 129+107=60/60=1.0, cub_018_Spotted_Catbird=0/0=nan
	Node name: 129+107, acc: 100.0, f1:100.0, samples: 60, 129+136=60/60=1.0, 107+151=0/0=nan
	Node name: 129+136, acc: 100.0, f1:100.0, samples: 60, 129+199=60/60=1.0, 136+085=0/0=nan
	Node name: 129+199, acc: 100.0, f1:100.0, samples: 60, 129+118=60/60=1.0, 199+186=0/0=nan
	Node name: 129+118, acc: 100.0, f1:100.0, samples: 60, 129+104=60/60=1.0, cub_118_House_Sparrow=0/0=nan
	Node name: 129

Test Epoch0: 100% 1/1 [00:00<00:00,  1.92it/s, L:0.055,LC:0.665, L_OVSP:-0.004, L_MASKL1:0.001, LA_PF:0.010, LT:-5.000, L_ORTH:0.005, LT_DESC:-5.000, L_OOD_ENT:-5.000, losses_used:MASK_PRUNING+MIN_CONT+AL_PF+KO+CL]


	Fine accuracy: 0.7
	Node name: root, acc: 90.0, f1:94.74, samples: 60, 129+024+067=54/60=0.9, 089+046=0/0=nan
	Node name: 129+024+067, acc: 90.0, f1:94.74, samples: 60, 129+065=54/60=0.9, 024+051=0/0=nan, 067+070=0/0=nan
	Node name: 129+065, acc: 100.0, f1:100.0, samples: 60, 129+192=60/60=1.0, 065+006=0/0=nan
	Node name: 129+192, acc: 100.0, f1:100.0, samples: 60, 129+043=60/60=1.0, 192+081=0/0=nan
	Node name: 129+043, acc: 93.33, f1:96.55, samples: 60, 129+018=56/60=0.93, 043+078=0/0=nan
	Node name: 129+018, acc: 93.33, f1:96.55, samples: 60, 129+107=56/60=0.93, cub_018_Spotted_Catbird=0/0=nan
	Node name: 129+107, acc: 93.33, f1:96.55, samples: 60, 129+136=56/60=0.93, 107+151=0/0=nan
	Node name: 129+136, acc: 80.0, f1:88.89, samples: 60, 129+199=48/60=0.8, 136+085=0/0=nan
	Node name: 129+199, acc: 90.0, f1:94.74, samples: 60, 129+118=0/0=nan, 199+186=54/60=0.9
	Node name: 199+186, acc: 96.67, f1:98.31, samples: 60, 199+150=0/0=nan, 186+185=58/60=0.97
	Node name: 186+185, acc: 0.0, f

## Leave out with mask pruning

In [19]:
# testloader

test_info, log_dict = test_pipnet(net, leave_out_loader, optimizer_net, optimizer_classifier, \
                                    scheduler_net, scheduler_classifier, criterion, 0, \
                                        args.epochs, device, pretrain=False, finetune=False, \
                                        test_loader_OOD=testloader_OOD, kernel_orth=args.kernel_orth == 'y', \
                                            tanh_desc=args.tanh_desc == 'y', align=args.align == 'y', uni=args.uni == 'y', align_pf=args.align_pf == 'y',\
                                            minmaximize=args.minmaximize == 'y', wandb_run=wandb_run, pretrain_epochs=args.epochs_pretrain, log=log, \
                                          args=args, apply_overspecificity_mask=True, leave_out_classes=leave_out_classes, path_prob_softmax_tau=1)

Test Epoch0: 100% 2/2 [00:03<00:00,  1.62s/it, L:1.455,LC:1.571, L_OVSP:-0.006, L_MASKL1:0.002, LA_PF:0.019, LT:-5.000, L_ORTH:0.002, LT_DESC:-5.000, L_OOD_ENT:0.550, losses_used:MASK_PRUNING+MIN_CONT+AL_PF+KO+CL+OOD_ENT]
/home/harishbabu/.conda/envs/hpnet4/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/harishbabu/.conda/envs/hpnet4/lib/python3.9/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


	Fine accuracy: 0.0
	Node name: root, acc: 0.0, f1:0.0, samples: 580, 129+024+067=0/580=0.0, 089+046=0/0=nan
	Node name: 129+024+067, acc: 24.48, f1:11.8, samples: 580, 129+065=0/418=0.0, 024+051=142/162=0.88, 067+070=0/0=nan
	Node name: 089+046, acc: inf, f1:inf, samples: 0, 089+090=0/0=nan, 046+087=0/0=nan
	Node name: 129+065, acc: 71.29, f1:59.34, samples: 418, 129+192=298/298=1.0, 065+006=0/120=0.0
	Node name: 024+051, acc: 37.04, f1:20.02, samples: 162, 024+031=0/102=0.0, 051+052=60/60=1.0
	Node name: 067+070, acc: inf, f1:inf, samples: 0, 067+068=0/0=nan, cub_070_Green_Violetear=0/0=nan
	Node name: 089+090, acc: inf, f1:inf, samples: 0, cub_089_Hooded_Merganser=0/0=nan, cub_090_Red_breasted_Merganser=0/0=nan
	Node name: 046+087, acc: inf, f1:inf, samples: 0, cub_046_Gadwall=0/0=nan, cub_087_Mallard=0/0=nan
	Node name: 129+192, acc: 20.13, f1:6.75, samples: 298, 129+043=0/238=0.0, 192+081=60/60=1.0
	Node name: 065+006, acc: 91.67, f1:95.65, samples: 120, 065+144=110/120=0.92, 006+

In [55]:
# print(net.module._root_proto_presence)
# print(net.module._root_proto_presence.shape)

In [56]:
# net.module._root_proto_presence[:, 0].shape

In [57]:
# torch.cat([net.module._root_proto_presence[:, 1].unsqueeze(-1), net.module._root_proto_presence[:, 0].unsqueeze(-1)], dim=-1)#.shape

In [25]:
# with torch.no_grad():
#     for node in root.nodes_with_children():
#         net.module._root_proto_presence
#         proto_presence = getattr(net.module, '_'+node.name+'_proto_presence')
#         reversed_proto_presence = torch.cat([proto_presence[:, 1].unsqueeze(-1), proto_presence[:, 0].unsqueeze(-1)], dim=-1)
#         setattr(net.module, '_'+node.name+'_proto_presence', \
#                 nn.Parameter(reversed_proto_presence))

#     test_info, log_dict = test_pipnet(net, leave_out_loader, optimizer_net, optimizer_classifier, \
#                                     scheduler_net, scheduler_classifier, criterion, 0, \
#                                         args.epochs, device, pretrain=False, finetune=False, \
#                                         test_loader_OOD=testloader_OOD, kernel_orth=args.kernel_orth == 'y', \
#                                             tanh_desc=args.tanh_desc == 'y', align=args.align == 'y', uni=args.uni == 'y', align_pf=args.align_pf == 'y',\
#                                             minmaximize=args.minmaximize == 'y', wandb_run=wandb_run, pretrain_epochs=args.epochs_pretrain, log=log, \
#                                           args=args, apply_overspecificity_mask=True)

In [19]:
torch.tensor([False, True, True]).sum()

tensor(2)